# Assignment 7 - Dialog Systems and Question Answering

- Student 1 - Luca Modica
- Student 2 - Hugo Alves Henriques E Silva

---

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from icecream import ic
import random


In [4]:


sns.set_style()
%matplotlib inline
np.random.seed(0)

Example from chatGPT

In [5]:
import requests

# API endpoints
weather_api_endpoint = "https://api.weather.com/forecast"
restaurant_api_endpoint = "https://api.yelp.com/v3/businesses/search"
transportation_api_endpoint = "https://api.transportation.com/schedule"

# API keys
weather_api_key = "your_weather_api_key"
restaurant_api_key = "your_restaurant_api_key"
transportation_api_key = "your_transportation_api_key"

def get_weather_forecast(city):
    params = {
        "apikey": weather_api_key,
        "city": city,
        "days": 1
    }
    response = requests.get(weather_api_endpoint, params=params)
    data = response.json()
    forecast = data["forecast"]["daily"]["summary"]
    return forecast

def find_restaurant(cuisine):
    headers = {
        "Authorization": f"Bearer {restaurant_api_key}"
    }
    params = {
        "term": cuisine,
        "location": "your_location"
    }
    response = requests.get(restaurant_api_endpoint, headers=headers, params=params)
    data = response.json()
    businesses = data["businesses"]
    if businesses:
        return random.choice(businesses)["name"]
    else:
        return None

def get_next_tram_bus(mode):
    headers = {
        "Authorization": f"Bearer {transportation_api_key}"
    }
    params = {
        "mode": mode,
        "location": "your_location"
    }
    response = requests.get(transportation_api_endpoint, headers=headers, params=params)
    data = response.json()
    schedule = data["schedule"]
    if schedule:
        return random.choice(schedule)["time"]
    else:
        return None

# Rest of the code remains the same as the previous implementation

In [6]:
import pandas as pd

df_restaurants = pd.read_csv("data/restaurants.csv")
df_restaurants.head()

,City,Restaurant,Street,Rating
0,London,Finnish,Lily Place 763,4.7
1,London,Croatian,Cherry Way 611,3.5
2,London,South African,Moonlight Avenue 181,4.5
3,Paris,Finnish,Peach Boulevard 803,4.5
4,Paris,Belgian,Rainbow Road 205,4.2


In [7]:
# Read the parquet file
df_weather = pd.read_parquet('data/daily_weather.parquet')
df_cities = pd.read_csv('data/cities.csv')

#remove entries from df_weather where the year is not 2023
df_weather = df_weather[df_weather['date'].dt.year == 2023]

# Display the head of the dataframe
print(df_weather.head())
print(df_cities.head())


     station_id city_name       date  season  avg_temp_c  min_temp_c  \
9332      41515  Asadabad 2023-01-01  Winter         5.7        -1.0   
9333      41515  Asadabad 2023-01-02  Winter         5.0        -1.4   
9334      41515  Asadabad 2023-01-03  Winter         4.9        -1.2   
9335      41515  Asadabad 2023-01-04  Winter         5.6        -1.4   
9336      41515  Asadabad 2023-01-05  Winter         7.9        -1.0   

      max_temp_c  precipitation_mm  snow_depth_mm  avg_wind_dir_deg  \
9332        13.0               0.0            NaN              81.0   
9333        11.8               0.0            NaN              72.0   
9334        11.6               0.0            NaN              95.0   
9335        12.7               0.0            NaN              99.0   
9336        15.0               0.0            NaN              73.0   

      avg_wind_speed_kmh  peak_wind_gust_kmh  avg_sea_level_pres_hpa  \
9332                 3.2                 NaN                  1021.6

In [8]:
#find temperature for the city of San Francisco
berlin = df_weather[df_weather['city_name'] == 'Berlin']
print(berlin)

      station_id city_name       date  season  avg_temp_c  min_temp_c  \
32034      10384    Berlin 2023-01-01  Winter        14.5        10.5   
32035      10384    Berlin 2023-01-02  Winter        13.1         9.1   
32036      10384    Berlin 2023-01-03  Winter         5.6         1.7   
32037      10384    Berlin 2023-01-04  Winter         7.1         2.8   
32038      10384    Berlin 2023-01-05  Winter         8.2         2.3   
...          ...       ...        ...     ...         ...         ...   
32275      10384    Berlin 2023-08-30  Summer        15.6        11.9   
32276      10384    Berlin 2023-08-31  Summer        16.2        12.0   
32277      10384    Berlin 2023-09-01  Autumn        16.7        12.2   
32278      10384    Berlin 2023-09-02  Autumn        17.6        13.3   
32279      10384    Berlin 2023-09-03  Autumn        18.5        13.5   

       max_temp_c  precipitation_mm  snow_depth_mm  avg_wind_dir_deg  \
32034        16.2               0.0            0.0 

In [9]:
weather_keywords = ["weather", "temperature", "forecast"]
restaurant_keywords = ["restaurant", "food", "cuisine"]
transportation_keywords = ["tram", "bus", "transportation"]


def search_keywords(phrase):
    for keyword in phrase.split():
        if keyword in weather_keywords:
            return "weather"
        elif keyword in restaurant_keywords:
            return "restaurant"
        elif keyword in transportation_keywords:
            return "transportation"
    return "I'm sorry, I don't have an answer for that, can you try to be more specific with the terminology?"

In [10]:

def weather_answer(phrase):
    city = ""
    country = ""
    for keyword in phrase.split():
        if keyword in df_cities['city_name'].values:
            city = keyword
        elif keyword in df_cities['country'].values:
            country = keyword
    
    if city == "" and country != "":
        new_question = input("Would you like to specify a city?")
        if new_question == "yes":
            city = input("Which city would you like to know the weather for?")
        else:
            # find the first city in the country
            city = df_cities[df_cities['country'] == country]['city_name'].values[0]
    
    if city != "":
        get_weather_forecast(city)
    else:
        print("Well... you either didn't specify a city or country, or I was not blessed with the knowledge of the one you are looking for...")
        print("Take it easy on me... I'm just a simple bot after all...")


def restaurant_answer(phrase):
    pass


def transportation_answer(phrase):
    pass

In [13]:
seasons = ['Spring', 'Summer', 'Autumn', 'Winter']

def get_weather_forecast(city):
    new_question = input("Would you like to know the weather for a specific season? Please answer yes or no.")
    if new_question == "yes":
        season = ""
        while season not in seasons:
            season = input("Which season would you like to know the weather for?")
        #get the average temperature for the season
        avg_temp = df_weather[(df_weather['city_name'] == city) & (df_weather['season'] == season)]['avg_temp_c'].mean()
        #get the year of the forecast
        date = df_weather[(df_weather['city_name'] == city) & (df_weather['season'] == season)]['date'].iloc[0]
        year = date.year
        print(f"The average temperature in {city} during {season} is {avg_temp} in {year}.")
    else:
        #get the average temperature for the whole year for the city
        avg_temp = df_weather[df_weather['city_name'] == city]['avg_temp_c'].mean()
        print(f"The average yearly temperature in {city} is {avg_temp}.")

    print(f"You seem to demonstrate an interest in {city}. Is there anything else you would like to know about it?")
    print("Sadly there is only so much I can help you with... I was purposefully designed to be a bit dumb, you see.")
    
        
    

In [14]:


while True:
    phrase = input("How can I help you today? ")
    if phrase == "exit":
        break
    else:
        context = search_keywords(phrase)
        if context == "weather":
            weather_answer(phrase)
        elif context == "restaurant":
            restaurant_answer(phrase)
        elif context == "transportation":
            transportation_answer(phrase)
            